In [2]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


**all-mpnet-base-v2** : All-round model tuned for many use-cases. Trained on a large and diverse dataset of over 1 billion training pairs.
cosine-similarity (util.cos_sim)

In [3]:
from sentence_transformers import SentenceTransformer
import json
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
with open('modified_faq_data.json', 'r') as file:
    data = json.load(file)

In [5]:
questions = [entry['question'] for entry in data]
answers = [entry['answer'] for entry in data]

In [6]:
question_embeddings = model.encode(questions)
answer_embeddings = model.encode(answers)

In [8]:
from sentence_transformers import util

# User's input question
user_input = input("Type your question: ")

# Encode the user's input
user_input_embedding = model.encode(user_input)

# Calculate cosine similarity between the user's input and all questions in the dataset
similarities = util.cos_sim(user_input_embedding, question_embeddings)[0]
most_similar_index = similarities.argmax()
most_similar_question = questions[most_similar_index]
most_similar_answer = answers[most_similar_index]

print(f"User's input: {user_input}")
print(f"Most similar question: {most_similar_question}")
print(f"Corresponding answer: {most_similar_answer}")

User's input: what is the difference between a commercial and an agent ? 
Most similar question: What is the difference between a Commercial Representative and an Agent?
Corresponding answer: An Agent is entitled to negotiate selling prices on behalf of the company it represents, while a Commercial Representative is not. MANE does not appoint Agents, only Commercial Representatives.


In [44]:
from sentence_transformers import SentenceTransformer, util
import json
from nltk.stem import WordNetLemmatizer
import os
import subprocess
import shlex  # Import shlex to split the command string into a list
# Load the lemmatizer for word normalization
lemmatizer = WordNetLemmatizer()

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load the FAQ data
with open('/home/mane/Mane-Project/Model/modified_faq_data.json', 'r') as file:
    data = json.load(file)

# Extract questions and answers from the data
questions = [entry['question'] for entry in data]
answers = [entry['answer'] for entry in data]

# Encode questions and answers
question_embeddings = model.encode(questions)
answer_embeddings = model.encode(answers)

# Set the working directory to the FastChat folder
fastchat_directory = '/home/mane/FastChat'
vicuna_model_path = 'lmsys/vicuna-7b-v1.5'
user_input = input("Type your question: ")

# Encode the user's input
user_input_embedding = model.encode(user_input)

# Calculate cosine similarity between the user's input and all questions in the dataset
similarities = util.cos_sim(user_input_embedding, question_embeddings)[0]
most_similar_index = similarities.argmax()
highest_similarity = similarities[most_similar_index]

# Set the threshold values
high_threshold = 0.8
medium_threshold = 0.5

# Construct the command to run Vicuna with the user's question

# Check the similarity against different thresholds
if highest_similarity >= high_threshold:
    # High similarity, directly provide the answer
    print(f"User's input: {user_input}")
    print(f"Most similar question: {questions[most_similar_index]}")
    print(f"Corresponding answer: {answers[most_similar_index]}")

elif medium_threshold <= highest_similarity < high_threshold:
    # Medium similarity, ask for clarification
    print(f"The question is similar to: {questions[most_similar_index]}")
    user_confirmation = input("Is this the same question? (yes/no): ").lower()

    if user_confirmation == 'yes':
        # User confirms, provide the answer
        print(f"User's input: {user_input}")
        print(f"Most similar question: {questions[most_similar_index]}")
        print(f"Corresponding answer: {answers[most_similar_index]}")
    else:
        # User denies, ask for clarification
        user_input += " " + input("Please provide more details or clarify your question: ")
        user_input_embedding = model.encode(user_input)
        similarities = util.cos_sim(user_input_embedding, question_embeddings)[0]
        most_similar_index = similarities.argmax()
        print(f"User's input after clarification: {user_input}")
        print(f"Most similar question after clarification: {questions[most_similar_index]}")
        print(f"Corresponding answer: {answers[most_similar_index]}")
else:
    # Low similarity, use Vicuna for answering
    print("Low similarity. Asking Vicuna for an answer...")

    # Set the working directory to the FastChat folder
    os.chdir('/home/mane/FastChat')

    # Construct the command to run Vicuna with the user's question
    vicuna_command = f'python3 -m fastchat.serve.cli --model {vicuna_model_path} "{user_input}"'
    try:
        # Use subprocess to run Vicuna and capture its output
        vicuna_answer = subprocess.check_output(vicuna_command, cwd=fastchat_directory, shell=True)
        print(f"Vicuna's answer: {vicuna_answer.decode('utf-8')}")
    except subprocess.CalledProcessError as e:
        print(f"Error while running Vicuna: {e}")
        vicuna_answer = "Error while running Vicuna"

    print(f"Vicuna's answer: {vicuna_answer}")

Low similarity. Asking Vicuna for an answer...


usage: cli.py [-h] [--model-path MODEL_PATH] [--revision REVISION]
              [--device {cpu,cuda,mps,xpu,npu}] [--gpus GPUS]
              [--num-gpus NUM_GPUS] [--max-gpu-memory MAX_GPU_MEMORY]
              [--dtype {float32,float16,bfloat16}] [--load-8bit]
              [--cpu-offloading] [--gptq-ckpt GPTQ_CKPT]
              [--gptq-wbits {2,3,4,8,16}] [--gptq-groupsize GPTQ_GROUPSIZE]
              [--gptq-act-order] [--awq-ckpt AWQ_CKPT] [--awq-wbits {4,16}]
              [--awq-groupsize AWQ_GROUPSIZE] [--enable-exllama]
              [--exllama-max-seq-len EXLLAMA_MAX_SEQ_LEN]
              [--exllama-gpu-split EXLLAMA_GPU_SPLIT] [--exllama-cache-8bit]
              [--enable-xft] [--xft-max-seq-len XFT_MAX_SEQ_LEN]
              [--xft-dtype {fp16,bf16,int8,bf16_fp16,bf16_int8}]
              [--conv-template CONV_TEMPLATE]
              [--conv-system-msg CONV_SYSTEM_MSG] [--temperature TEMPERATURE]
              [--repetition_penalty REPETITION_PENALTY]
              [--

Error while running Vicuna: Command 'python3 -m fastchat.serve.cli --model lmsys/vicuna-7b-v1.5 "how are you ?"' returned non-zero exit status 2.
Vicuna's answer: Error while running Vicuna
